# Propositionalization: Traffic near Dodgers' stadium

In this notebook, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.

Summary:

- Prediction type: __Regression model__
- Domain: __Transportation__
- Prediction target: __traffic volume__ 
- Source data: __Univariate time series__
- Population size: __47497__

# Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

In this notebook, we use traffic data that was collected for the Glendale on ramp for the 101 North freeway in Los Angeles. For further details about the data set refer to [the full notebook](../dodgers.ipynb).

# Analysis

## Table of contents

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Comparison](#3.-Comparison)

Let's get started with the analysis and set-up your session:

In [1]:
import datetime
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pathlib import Path

import sys
from urllib import request

import pandas as pd

In [2]:
parent = Path(os.getcwd()).parent.as_posix()

if parent not in sys.path:
   sys.path.append(parent) 

from utils import Benchmark, FTTimeSeriesBuilder, TSFreshBuilder

In [3]:
import getml

getml.engine.launch(home_directory=Path.home(), allow_remote_ips=True, token='token')
getml.engine.set_project("dodgers")

Launching ./getML --allow-push-notifications=true --allow-remote-ips=true --home-directory=/home/getml --in-memory=true --install=false --launch-browser=true --log=false --token=token in /home/getml/.getML/getml-1.4.0-x64-linux...
Launched the getML engine. The log output will be stored in /home/getml/.getML/logs/20240221161224.log.
Loading pipelines... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          

Connected to project 'dodgers'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UC Irvine Machine Learning repository:

In [4]:
fname = "Dodgers.data"

if not os.path.exists(fname):
    fname, res = request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/"
        + fname,
        fname,
    )

data_full_pandas = pd.read_csv(fname, header=None)
data_full_pandas.columns = ["ds", "y"]

In [5]:
data_full_pandas["ds"] = [
    datetime.datetime.strptime(dt, "%m/%d/%Y %H:%M") for dt in data_full_pandas["ds"]
]

In [6]:
data_full_pandas

,ds,y
0,2005-04-10 00:00:00,-1
1,2005-04-10 00:05:00,-1
2,2005-04-10 00:10:00,-1
3,2005-04-10 00:15:00,-1
4,2005-04-10 00:20:00,-1
...,...,...
50395,2005-10-01 23:35:00,-1
50396,2005-10-01 23:40:00,-1
50397,2005-10-01 23:45:00,-1
50398,2005-10-01 23:50:00,-1


### 1.2 Prepare data for getML

In [7]:
data_full = getml.data.DataFrame.from_pandas(data_full_pandas, "data_full")

In [8]:
data_full.set_role("y", getml.data.roles.target)
data_full.set_role("ds", getml.data.roles.time_stamp)

In [9]:
data_full

name,ds,y
role,time_stamp,target
unit,"time stamp, comparison only",
0,2005-04-10,-1
1,2005-04-10 00:05:00,-1
2,2005-04-10 00:10:00,-1
3,2005-04-10 00:15:00,-1
4,2005-04-10 00:20:00,-1
,...,...
50395,2005-10-01 23:35:00,-1
50396,2005-10-01 23:40:00,-1


In [10]:
split = getml.data.split.time(
    population=data_full, time_stamp="ds", test=getml.data.time.datetime(2005, 8, 20)
)
split

,
0,train
1,train
2,train
3,train
4,train
,...


### 1.3 Define relational model

To start with relational learning, we need to specify the data model. We manually replicate the appropriate time series structure by setting time series related join conditions (`horizon`, `memory` and `allow_lagged_targets`). This is done abstractly using [Placeholders](https://docs.getml.com/latest/user_guide/data_model/data_model.html#placeholders)

The data model consists of two tables:
* __Population table__ `traffic_{test/train}`: holds target and the contemporarily available time-based components
* __Peripheral table__ `traffic`: same table as the population table
* Join between both placeholders specifies (`horizon`) to prevent leaks and (`memory`) that keeps the computations feasible

In [11]:
# 1. The horizon is 1 hour (we predict the traffic volume in one hour).
# 2. The memory is 2 hours, so we allow the algorithm to
#    use information from up to 2 hours ago.
# 3. We allow lagged targets. Thus, the algorithm can
#    identify autoregressive processes.

time_series = getml.data.TimeSeries(
    population=data_full,
    alias="population",
    split=split,
    time_stamps="ds",
    horizon=getml.data.time.hours(1),
    memory=getml.data.time.hours(2),
    lagged_targets=True,
)

time_series

data model

  population:
    columns:
    - y: target
    - ds: time_stamp

    joins:
    - right: 'data_full'
      time_stamps: (population.ds, data_full.ds)
      relationship: 'many-to-many'
      memory: 7200.0
      horizon: 3600.0
      lagged_targets: True

  data_full:
    columns:
    - y: target
    - ds: time_stamp


container

  population
      subset   name         rows   type
  0   test     data_full   12384   View
  1   train    data_full   38016   View

  peripheral
      name         rows   type     
  0   data_full   50400   DataFrame

## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 Propositionalization with getML's FastProp

In [12]:
seasonal = getml.preprocessors.Seasonal()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
)

__Build the pipeline__

In [13]:
pipe_fp_fl = getml.pipeline.Pipeline(
    preprocessors=[seasonal],
    feature_learners=[fast_prop],
    data_model=time_series.data_model,
    tags=["feature learning", "fastprop"],
)

pipe_fp_fl

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['data_full'],
         predictors=[],
         preprocessors=['Seasonal'],
         share_selected_features=0.5,
         tags=['feature learning', 'fastprop'])

In [14]:
pipe_fp_fl.check(time_series.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

OK.


In [15]:
benchmark = Benchmark()

In [16]:
with benchmark("fastprop"):
    pipe_fp_fl.fit(time_series.train)
    fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Trying 526 features... 100% |██████████| [elapsed: 00:06, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:6.317863

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:03, remaining: 00:00]          



In [17]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



In [18]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [19]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:09, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:9.613381



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

In [20]:
pipe_fp_pr.score(fastprop_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2024-02-21 16:12:55,fastprop_train,y,5.4188,7.5347,0.699
1,2024-02-21 16:12:55,fastprop_test,y,5.6151,7.8243,0.6747


### 2.2 Propositionalization with featuretools

In [21]:
data_train = time_series.train.population.to_df("data_train")
data_test = time_series.test.population.to_df("data_test")

In [22]:
dfs_pandas = {}

for df in getml.project.data_frames:
    dfs_pandas[df.name] = df.to_pandas()
    dfs_pandas[df.name]["id"] = 1

In [23]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(hours=1),
    memory=pd.Timedelta(hours=2),
    column_id="id",
    time_stamp="ds",
    target="y",
    allow_lagged_targets=True,
)

In [24]:
with benchmark("featuretools"):
    featuretools_train = ft_builder.fit(dfs_pandas["data_train"])

featuretools_test = ft_builder.transform(dfs_pandas["data_test"])

featuretools: Trying features...
Selecting the best out of 118 features...
Time taken: 0h:9m:19.75259



In [25]:
df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=data_train.roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=data_train.roles
)

In [26]:
df_featuretools_train.set_role(
    df_featuretools_train.roles.unused, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.roles.unused, getml.data.roles.numerical
)

In [27]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [28]:
pipe_ft_pr.check(df_featuretools_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.


,type,label,message
0,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'id' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


In [29]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:04, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:4.092266



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [30]:
pipe_ft_pr.score(df_featuretools_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2024-02-21 16:25:31,featuretools_train,y,5.4482,7.568,0.6962
1,2024-02-21 16:25:31,featuretools_test,y,6.0863,8.5009,0.6498


### 2.3 Propositionalization with tsfresh

In [31]:
tsfresh_builder = TSFreshBuilder(
    num_features=200,
    horizon=20,
    memory=60,
    column_id="id",
    time_stamp="ds",
    target="y",
    allow_lagged_targets=True,
)

In [32]:
with benchmark("tsfresh"):
    tsfresh_train = tsfresh_builder.fit(dfs_pandas["data_train"])

tsfresh_test = tsfresh_builder.transform(dfs_pandas["data_test"])

Feature Extraction: 100%|██████████| 40/40 [00:08<00:00,  4.71it/s]


Selecting the best out of 13 features...
Time taken: 0h:0m:46.114942



Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 12.04it/s]


In [33]:
df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=data_train.roles
)

df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=data_train.roles
)

In [34]:
df_tsfresh_train.set_role(df_tsfresh_train.roles.unused, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.roles.unused, getml.data.roles.numerical)

In [35]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [36]:
pipe_tsf_pr.fit(df_tsfresh_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

The pipeline check generated 0 issues labeled INFO and 1 issues labeled WARNING.
To see the issues in full, run .check() on the pipeline.

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:1.790984



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [37]:
pipe_tsf_pr.score(df_tsfresh_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2024-02-21 16:26:34,tsfresh_train,y,6.3146,8.2348,0.6418
1,2024-02-21 16:26:34,tsfresh_test,y,6.7886,8.9134,0.5778


## 3. Comparison

In [38]:
num_features = dict(
    fastprop=526,
    featuretools=59,
    tsfresh=12,
)

runtime_per_feature = [
    benchmark.runtimes["fastprop"] / num_features["fastprop"],
    benchmark.runtimes["featuretools"] / num_features["featuretools"],
    benchmark.runtimes["tsfresh"] / num_features["tsfresh"],
]

features_per_second = [1.0 / r.total_seconds() for r in runtime_per_feature]

normalized_runtime_per_feature = [
    r / runtime_per_feature[0] for r in runtime_per_feature
]

comparison = pd.DataFrame(
    dict(
        runtime=[
            benchmark.runtimes["fastprop"],
            benchmark.runtimes["featuretools"],
            benchmark.runtimes["tsfresh"],
        ],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        normalized_runtime=[
            1,
            benchmark.runtimes["featuretools"] / benchmark.runtimes["fastprop"],
            benchmark.runtimes["tsfresh"] / benchmark.runtimes["fastprop"],
        ],
        normalized_runtime_per_feature=normalized_runtime_per_feature,
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [39]:
comparison

,runtime,num_features,features_per_second,normalized_runtime,normalized_runtime_per_feature,rsquared,rmse,mae
getML: FastProp,0 days 00:00:09.406415,526,55.919029,1.000000,1.000000,0.674740,7.824273,5.615138
featuretools,0 days 00:09:19.754041,59,0.105403,59.507691,530.523794,0.649768,8.500887,6.086277
tsfresh,0 days 00:00:46.115063,12,0.260219,4.902512,214.892468,0.577811,8.913408,6.788610


In [40]:
# export for further use
comparison.to_csv("comparisons/dodgers.csv")